Makings Spuds out of smaller parts

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

Dict{Int64, Function} with 10 entries:
  50  => fx_melodious
  20  => fx_reciprocating
  110 => fx_otaku
  10  => fx_imitative
  90  => fx_iconoclast
  30  => fx_romantic
  120 => fx_sincere
  130 => fx_cowardly
  40  => fx_bibliophile
  100 => fx_intoxicating

In [4]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, utb)
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

In [10]:
function sample_library(ss_prob, as, n_init = 1000000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for a1 in as
        for h in hrange
            for f in frange
                for l in lrange
                    for p in prange
                        for r in rrange
                            for s in srange
                                if rand() <= ss_prob
                                    ff = Spud("", h,f,l,p,r,s,a1)
                                    ff_i += 1
                                    ffs[ff_i] = ff
                                end
                            end
                        end                        
                    end                        
                end
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

sample_library (generic function with 2 methods)

In [11]:
struct NashEquilibrium2p
    w1::Array{Float64}
    w2::Array{Float64}
    v::Float64
end


function payoffs2(set1::Array{Spud}, set2::Array{Spud})::Array{Int64}
    n1 = length(set1)
    n2 = length(set2)
    payoffs = Array{Int64}(undef, (n1, n2))
    for i in 1:n1
        for j in 1:n2
            payoffs[i,j] = eval_battle(set1[i], set2[j])
        end
    end
    return payoffs
end

function ffp2(set1::Array{Spud}, set2::Array{Spud}, nits::Int64)::NashEquilibrium2p
    n1 = length(set1)
    n2 = length(set2)
    payoffs = payoffs2(set1, set2)
    v1 = [1 for i in 1:n1]
    v2 = [1 for i in 1:n2]
    s1 = [sum(payoffs[i, :] .* v2) for i in 1:n1]
    s2 = [sum(payoffs[:, i] .* v1) for i in 1:n2]
    for it in 1:nits
        cands1 = findall(s1 .== maximum(s1))
        i1 = rand(cands1)
        cands2 = findall(s2 .== minimum(s2))
        i2 = rand(cands2)
        v1[i1] += 1
        s2 = s2 .+ payoffs[i1, :]
        v2[i2] += 1
        s1 = s1 .+ payoffs[:, i2]
    end
    w1 = v1./sum(v1)
    w2 = v2./sum(v2)
    val = sum([payoffs[i, j] * w1[i] * w2[j] for i in 1:n1 for j in 1:n2])
    return NashEquilibrium2p(w1, w2, val)
end

function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        wc = 2 .* wins .+ counts
        ind_winners = findall(wc .== maximum(wc))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end

ffp (generic function with 1 method)

## Battle code

In [12]:
mutable struct BattleView
    round::Int64
    own::Array{Spud}
    owncount::Array{Int64}
    deployed::Array{Bool}
    obs::Array{Spud}
    obscount::Array{Int64}
end

mutable struct AiPlayer
    genteam::Function
    pick::Function
    create::Function
end

In [13]:
n_copy = 3

function initialize_game(team1::Array{Spud}, team2::Array{Spud})::Array{BattleView}
    teams = [team1, team2]
    bvs = Array{BattleView}(undef, 0)
    for team in teams
        bv = BattleView(0, team, [n_copy for i in 1:length(team)], [false for i in 1:length(team)], [], [])
        append!(bvs, [bv])
    end
    return bvs
end

function observe_fighters(game::Array{BattleView}, choices::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].deployed[choices[i]] = true
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                0
            else
                game[j].obs = append!(game[j].obs, [ff])
                game[j].obscount = append!(game[j].obscount, [game[i].owncount[choices[i]]])
            end
        end
    end
    return game
end

function modify_fighter_counts(game::Array{BattleView}, choices::Array{Int64}, mod::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].owncount[choices[i]] = game[i].owncount[choices[i]] + mod[i]
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                ind = findall(x -> x==ff, game[j].obs)[1]
                game[j].obscount[ind] = game[i].owncount[choices[i]]
            else
                0
            end
        end
    end
    return game
end

function play_combat(game::Array{BattleView}, choices::Array{Int64}, verbose::Bool = false)::Array{BattleView}
    round = game[1].round + 1
    fighters = [game[i].own[choices[i]] for i in 1:2]
    for i in 1:2
        @assert game[i].owncount[choices[i]] >= 0
    end
    if verbose
        print("Round ")
        print(round)
        print(": #A")
        print(choices[1])
        print(" ")
        print(fighters[1])
        print(" vs #B")
        print(choices[2])
        print(" ")
        println(fighters[2])
    end
    outcome = eval_battle(fighters[1], fighters[2])
    if verbose
        if outcome == 0
            println("Tied!")
        else
            if outcome == 1
                print(fighters[1])
            end
            if outcome == -1
                print(fighters[2])
            end
            println(" wins!")
        end
    end
    game = observe_fighters(game, choices)
    if outcome > -1
        game = modify_fighter_counts(game, choices, [0, -1])
    end
    if outcome < 1
        game = modify_fighter_counts(game, choices, [-1, 0])
    end
    for i in 1:2
        game[i].round = round
    end
    return game
end

function play_add_spud(game::Array{BattleView}, player::Int64, ff::Spud, verbose::Bool = false)::Array{BattleView}
    game[player].own = append!(game[player].own, [ff])
    game[player].owncount = append!(game[player].owncount, [n_copy])
    game[player].deployed = append!(game[player].deployed, [false])
    if verbose
        print("Player ")
        print(player)
        print(" adds #")
        print(["A", "B"][player])
        print(length(game[player].own))
        print(" ")
        println(ff)
    end
    return game
end

function play_is_game_over(game::Array{BattleView})::Bool
    for bv in game
        if sum(bv.owncount) == 0
            return true
        end
    end
    return false
end

play_is_game_over (generic function with 1 method)

In [14]:
function generate_rand_team(env, counts, nteam)
    cc = cumsum(counts)
    team = Array{Spud}(undef, nteam)
    for i in 1:nteam
        tmp = rand(1:sum(counts))
        ind = sum(cc .< tmp)+1
        team[i] = env[ind]
    end
    return team
end

function ai0_pick(a::BattleView)::Int64
    inds = findall(a.owncount .> 0)
    return rand(inds)
end

function ai0_create(a::BattleView)::Spud
    return generate_rand_team(nash_env, counts, 1)[1]
end

function ai1_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    cands = nash_env[ebs .== maximum(ebs)]
    return rand(cands)
end

function ai2_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    if maximum(ebs) == length(ops)
        cands = nash_env[ebs .== maximum(ebs)]
        return rand(cands)
    else
        ebs2 = [eval_battle_list(ff, ops) for ff in lib]
        if maximum(ebs2) > maximum(ebs)
            cands = lib[ebs2 .== maximum(ebs2)]
            return rand_rename(rand(cands))
        else
            cands = nash_env[ebs .== maximum(ebs)]
            return rand(cands)
        end
    end
end


ai2_create (generic function with 1 method)

In [15]:
# picks Spud most likely to win
function ai_greedy_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    ind = findall(a.owncount .> 0)[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

ai_greedy_pick (generic function with 1 method)

In [16]:
# picks from spuds already deployed
function ai_conservative_pick(a::BattleView)::Int64
    own = a.own[a.owncount .> 0 .&& a.deployed]
    owninds = findall(a.owncount .> 0 .&& a.deployed)
    if length(own) == 0
        # pick greedy
        return ai_greedy_pick(a)
    end
    ops = a.obs[a.obscount .> 0]
    res = ffp2(own, ops, 100)
    ind = owninds[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

ai_conservative_pick (generic function with 1 method)

In [17]:
# picks Spud based on 1-round nash
function ai_myopic_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    cc = cumsum(res.w1)
    ind = sum(cc .< rand()) + 1
    return findall(a.owncount .> 0)[ind]
end

ai_myopic_pick (generic function with 1 method)

In [18]:
creation_rounds = [5, 10, 15]

function play_game(players::Array{AiPlayer}, verbose::Bool = true)::Int64
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    team1 = ai1.genteam()
    team2 = ai2.genteam()
    game = initialize_game(team1, team2)
    while !play_is_game_over(game)
        # play a round
        choice1 = ai1.pick(game[1])
        choice2 = ai2.pick(game[2])
        game = play_combat(game, [choice1, choice2], verbose)
        if verbose
            for i in 1:2
                println(game[i].owncount)
            end
        end
        if game[1].round in creation_rounds
            for i in 1:2
                ff = ais[i].create(game[i])
                game = play_add_spud(game, i, ff, verbose)
            end
        end
    end
    if sum(game[1].owncount) == 0
        if sum(game[2].owncount) == 0
            return 0
        else
            return -1
        end
    end
    return 1
end

play_game (generic function with 2 methods)

In [19]:
# ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ais = [ai1, ai2]
# play_game(ais, true)

In [206]:
#@time lib = sample_library(1.01, [999]);

In [21]:
function filter_nondominated(as::Array{Spud})::Array{Spud}
    df = spuds_to_df(as)
    mat = Array{Int64}(undef, (length(as), 6))
    mat[:, 1] = df.h
    mat[:, 2] = df.f
    mat[:, 3] = df.l
    mat[:, 4] = df.p
    mat[:, 5] = df.r
    mat[:, 6] = df.s;
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:6
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

function spud_to_vec(a::Spud)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

spud_to_vec (generic function with 1 method)

## Analyzing part costs

In [27]:
using MAT
res = matread("scores_sq300.mat");

In [28]:
scores = res["scores"];

In [52]:
idxa = [ones(Int64, (9,9,9,9,9,9)) for i in 1:6]
for i in 1:9
    idxa[1][i,:,:,:,:,:] .= i
    idxa[2][:,i,:,:,:,:] .= i
    idxa[3][:,:,i,:,:,:] .= i
    idxa[4][:,:,:,i,:,:] .= i
    idxa[5][:,:,:,:,i,:] .= i
    idxa[6][:,:,:,:,:,i] .= i
end

In [29]:
#inds = findall(scores .> 99 .&& scores .<= 100);

In [30]:
#nash_env = [Spud("",v[1],v[2],v[3],v[4],v[5],v[6],999) for v in inds];

#### increasing H by 1

In [31]:
# cost of increasing H by 1
base_cost = scores[1:8,:,:,:,:,:];
diff = scores[2:9,:,:,:,:,:] - scores[1:8,:,:,:,:,:];

In [32]:
maximum(diff)

35

In [33]:
findall(diff .== maximum(diff))

28-element Vector{CartesianIndex{6}}:
 CartesianIndex(5, 9, 4, 1, 5, 1)
 CartesianIndex(5, 8, 5, 1, 5, 1)
 CartesianIndex(5, 7, 7, 1, 5, 1)
 CartesianIndex(5, 6, 8, 1, 5, 1)
 CartesianIndex(5, 9, 4, 1, 6, 1)
 CartesianIndex(5, 7, 6, 1, 6, 1)
 CartesianIndex(5, 8, 6, 1, 6, 1)
 CartesianIndex(5, 6, 7, 1, 6, 1)
 CartesianIndex(5, 8, 4, 1, 7, 1)
 CartesianIndex(5, 7, 5, 1, 7, 1)
 CartesianIndex(5, 7, 6, 1, 7, 1)
 CartesianIndex(5, 6, 8, 1, 7, 1)
 CartesianIndex(5, 6, 9, 1, 7, 1)
 ⋮
 CartesianIndex(5, 7, 6, 1, 9, 1)
 CartesianIndex(5, 6, 7, 1, 4, 2)
 CartesianIndex(5, 5, 9, 1, 4, 2)
 CartesianIndex(5, 6, 7, 2, 4, 2)
 CartesianIndex(5, 6, 8, 2, 4, 2)
 CartesianIndex(5, 5, 9, 2, 4, 2)
 CartesianIndex(5, 6, 7, 1, 5, 2)
 CartesianIndex(5, 6, 8, 1, 5, 2)
 CartesianIndex(5, 5, 8, 1, 6, 2)
 CartesianIndex(5, 5, 9, 1, 6, 2)
 CartesianIndex(5, 5, 8, 2, 6, 2)
 CartesianIndex(5, 5, 9, 1, 7, 2)

In [34]:
scores[5, 9, 4, 1, 5, 1]

77

In [35]:
scores[6, 9, 4, 1, 5, 1]

112

In [36]:
scores[5, 5, 9, 1, 6, 2]

88

In [37]:
scores[6, 5, 9, 1, 6, 2]

123

In [40]:
# cost of increasing H by 1
# excluding when cost > 100
base_cost = scores[1:8,:,:,:,:,:];
end_cost = scores[2:9,:,:,:,:,:]
diff = end_cost - base_cost
diff[end_cost .> 100] .= -1;

In [41]:
println(maximum(diff))
findall(diff .== maximum(diff))

34


20-element Vector{CartesianIndex{6}}:
 CartesianIndex(5, 8, 4, 1, 3, 1)
 CartesianIndex(5, 7, 5, 1, 4, 1)
 CartesianIndex(5, 7, 5, 1, 5, 1)
 CartesianIndex(5, 6, 6, 1, 5, 1)
 CartesianIndex(5, 7, 4, 1, 6, 1)
 CartesianIndex(5, 6, 6, 1, 6, 1)
 CartesianIndex(5, 5, 7, 1, 6, 1)
 CartesianIndex(5, 6, 5, 1, 7, 1)
 CartesianIndex(5, 5, 7, 1, 7, 1)
 CartesianIndex(5, 7, 4, 1, 8, 1)
 CartesianIndex(5, 7, 4, 1, 9, 1)
 CartesianIndex(5, 7, 4, 1, 3, 2)
 CartesianIndex(5, 6, 5, 1, 4, 2)
 CartesianIndex(5, 7, 4, 1, 5, 2)
 CartesianIndex(5, 6, 5, 1, 5, 2)
 CartesianIndex(5, 6, 5, 1, 6, 2)
 CartesianIndex(5, 6, 5, 1, 7, 2)
 CartesianIndex(5, 7, 4, 1, 2, 3)
 CartesianIndex(5, 6, 5, 1, 3, 3)
 CartesianIndex(5, 6, 5, 1, 1, 4)

In [46]:
[rand(findall(diff .== 33)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(5, 8, 3, 1, 7, 1)
 CartesianIndex(5, 7, 5, 1, 2, 1)
 CartesianIndex(5, 5, 6, 4, 4, 2)
 CartesianIndex(5, 6, 5, 1, 6, 1)
 CartesianIndex(5, 7, 4, 2, 2, 2)
 CartesianIndex(5, 5, 6, 1, 1, 5)
 CartesianIndex(5, 7, 5, 1, 1, 2)
 CartesianIndex(5, 7, 5, 2, 2, 1)
 CartesianIndex(5, 7, 4, 1, 1, 4)
 CartesianIndex(4, 4, 9, 1, 3, 6)

In [47]:
[rand(findall(diff .== 32)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(5, 8, 2, 1, 1, 3)
 CartesianIndex(4, 4, 8, 2, 2, 6)
 CartesianIndex(5, 7, 4, 1, 3, 1)
 CartesianIndex(5, 5, 6, 1, 1, 4)
 CartesianIndex(4, 7, 5, 1, 1, 6)
 CartesianIndex(4, 6, 5, 1, 2, 8)
 CartesianIndex(5, 6, 4, 1, 3, 4)
 CartesianIndex(4, 1, 9, 4, 8, 5)
 CartesianIndex(5, 7, 3, 1, 2, 4)
 CartesianIndex(5, 5, 5, 1, 7, 3)

In [48]:
[rand(findall(diff .== 20)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(3, 2, 9, 4, 8, 3)
 CartesianIndex(3, 6, 5, 3, 4, 7)
 CartesianIndex(2, 4, 9, 4, 5, 6)
 CartesianIndex(5, 1, 5, 1, 9, 2)
 CartesianIndex(5, 2, 4, 6, 5, 1)
 CartesianIndex(5, 1, 5, 4, 4, 7)
 CartesianIndex(4, 3, 6, 1, 4, 2)
 CartesianIndex(4, 4, 4, 3, 4, 4)
 CartesianIndex(3, 7, 8, 2, 3, 2)
 CartesianIndex(4, 3, 5, 5, 4, 4)

In [49]:
[rand(findall(diff .== 10)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(2, 8, 2, 9, 2, 2)
 CartesianIndex(3, 7, 3, 7, 3, 1)
 CartesianIndex(3, 3, 5, 5, 7, 2)
 CartesianIndex(2, 1, 6, 8, 9, 1)
 CartesianIndex(6, 3, 2, 1, 9, 5)
 CartesianIndex(6, 2, 4, 6, 5, 9)
 CartesianIndex(8, 1, 5, 9, 2, 2)
 CartesianIndex(6, 2, 2, 5, 1, 5)
 CartesianIndex(2, 8, 1, 1, 7, 6)
 CartesianIndex(7, 3, 4, 9, 4, 4)

In [59]:
println(maximum(diff[idxa[1][1:8,:,:,:,:,:] .== 1]))
findall(diff .== maximum(diff[idxa[1][1:8,:,:,:,:,:] .== 1]) .&& idxa[1][1:8,:,:,:,:,:] .== 1)

19


16-element Vector{CartesianIndex{6}}:
 CartesianIndex(1, 3, 9, 9, 1, 8)
 CartesianIndex(1, 2, 9, 1, 9, 8)
 CartesianIndex(1, 3, 9, 8, 1, 9)
 CartesianIndex(1, 4, 9, 8, 1, 9)
 CartesianIndex(1, 3, 9, 1, 8, 9)
 CartesianIndex(1, 3, 9, 2, 8, 9)
 CartesianIndex(1, 4, 9, 2, 8, 9)
 CartesianIndex(1, 1, 9, 1, 9, 9)
 CartesianIndex(1, 2, 9, 1, 9, 9)
 CartesianIndex(1, 3, 9, 1, 9, 9)
 CartesianIndex(1, 1, 9, 2, 9, 9)
 CartesianIndex(1, 2, 9, 2, 9, 9)
 CartesianIndex(1, 3, 9, 2, 9, 9)
 CartesianIndex(1, 1, 9, 3, 9, 9)
 CartesianIndex(1, 2, 9, 3, 9, 9)
 CartesianIndex(1, 3, 9, 3, 9, 9)

In [62]:
j = 2
println(maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]))
findall(diff .== maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]) .&& idxa[1][1:8,:,:,:,:,:] .== j)

22


2-element Vector{CartesianIndex{6}}:
 CartesianIndex(2, 2, 9, 1, 9, 9)
 CartesianIndex(2, 2, 9, 2, 9, 9)

In [63]:
j = 3
println(maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]))
findall(diff .== maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]) .&& idxa[1][1:8,:,:,:,:,:] .== j)

28


9-element Vector{CartesianIndex{6}}:
 CartesianIndex(3, 1, 9, 7, 7, 9)
 CartesianIndex(3, 1, 9, 8, 7, 9)
 CartesianIndex(3, 1, 9, 9, 7, 9)
 CartesianIndex(3, 1, 9, 9, 8, 9)
 CartesianIndex(3, 1, 9, 1, 9, 9)
 CartesianIndex(3, 1, 9, 3, 9, 9)
 CartesianIndex(3, 1, 9, 5, 9, 9)
 CartesianIndex(3, 1, 9, 7, 9, 9)
 CartesianIndex(3, 1, 9, 9, 9, 9)

In [64]:
j = 4
println(maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]))
findall(diff .== maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]) .&& idxa[1][1:8,:,:,:,:,:] .== j)

33


22-element Vector{CartesianIndex{6}}:
 CartesianIndex(4, 4, 9, 2, 4, 5)
 CartesianIndex(4, 6, 7, 1, 1, 6)
 CartesianIndex(4, 5, 8, 1, 1, 6)
 CartesianIndex(4, 5, 9, 1, 1, 6)
 CartesianIndex(4, 5, 8, 2, 1, 6)
 CartesianIndex(4, 4, 9, 1, 3, 6)
 CartesianIndex(4, 6, 5, 1, 1, 7)
 CartesianIndex(4, 6, 6, 1, 1, 7)
 CartesianIndex(4, 5, 7, 1, 1, 7)
 CartesianIndex(4, 5, 8, 1, 1, 7)
 CartesianIndex(4, 4, 9, 1, 1, 7)
 CartesianIndex(4, 5, 7, 2, 1, 7)
 CartesianIndex(4, 4, 8, 3, 1, 7)
 CartesianIndex(4, 5, 7, 1, 2, 7)
 CartesianIndex(4, 6, 5, 1, 1, 8)
 CartesianIndex(4, 5, 6, 1, 1, 8)
 CartesianIndex(4, 5, 7, 1, 1, 8)
 CartesianIndex(4, 4, 8, 1, 1, 8)
 CartesianIndex(4, 3, 9, 1, 1, 8)
 CartesianIndex(4, 5, 6, 1, 2, 8)
 CartesianIndex(4, 1, 9, 2, 2, 8)
 CartesianIndex(4, 5, 6, 1, 1, 9)

In [65]:
j = 5
println(maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]))
findall(diff .== maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]) .&& idxa[1][1:8,:,:,:,:,:] .== j)

34


20-element Vector{CartesianIndex{6}}:
 CartesianIndex(5, 8, 4, 1, 3, 1)
 CartesianIndex(5, 7, 5, 1, 4, 1)
 CartesianIndex(5, 7, 5, 1, 5, 1)
 CartesianIndex(5, 6, 6, 1, 5, 1)
 CartesianIndex(5, 7, 4, 1, 6, 1)
 CartesianIndex(5, 6, 6, 1, 6, 1)
 CartesianIndex(5, 5, 7, 1, 6, 1)
 CartesianIndex(5, 6, 5, 1, 7, 1)
 CartesianIndex(5, 5, 7, 1, 7, 1)
 CartesianIndex(5, 7, 4, 1, 8, 1)
 CartesianIndex(5, 7, 4, 1, 9, 1)
 CartesianIndex(5, 7, 4, 1, 3, 2)
 CartesianIndex(5, 6, 5, 1, 4, 2)
 CartesianIndex(5, 7, 4, 1, 5, 2)
 CartesianIndex(5, 6, 5, 1, 5, 2)
 CartesianIndex(5, 6, 5, 1, 6, 2)
 CartesianIndex(5, 6, 5, 1, 7, 2)
 CartesianIndex(5, 7, 4, 1, 2, 3)
 CartesianIndex(5, 6, 5, 1, 3, 3)
 CartesianIndex(5, 6, 5, 1, 1, 4)

In [66]:
j = 6
println(maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]))
findall(diff .== maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]) .&& idxa[1][1:8,:,:,:,:,:] .== j)

26


7-element Vector{CartesianIndex{6}}:
 CartesianIndex(6, 6, 4, 1, 1, 1)
 CartesianIndex(6, 6, 4, 1, 2, 1)
 CartesianIndex(6, 6, 4, 2, 2, 1)
 CartesianIndex(6, 6, 4, 1, 4, 1)
 CartesianIndex(6, 6, 4, 1, 1, 2)
 CartesianIndex(6, 6, 4, 1, 2, 2)
 CartesianIndex(6, 6, 4, 1, 3, 2)

In [67]:
j = 7
println(maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]))
findall(diff .== maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]) .&& idxa[1][1:8,:,:,:,:,:] .== j)

21


10-element Vector{CartesianIndex{6}}:
 CartesianIndex(7, 6, 3, 1, 1, 1)
 CartesianIndex(7, 6, 3, 2, 1, 1)
 CartesianIndex(7, 6, 2, 4, 1, 1)
 CartesianIndex(7, 6, 2, 2, 3, 1)
 CartesianIndex(7, 6, 2, 1, 4, 1)
 CartesianIndex(7, 6, 1, 1, 1, 2)
 CartesianIndex(7, 6, 2, 1, 1, 2)
 CartesianIndex(7, 6, 2, 1, 2, 2)
 CartesianIndex(7, 6, 1, 1, 1, 3)
 CartesianIndex(7, 5, 4, 1, 2, 3)

In [68]:
j = 8
println(maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]))
findall(diff .== maximum(diff[idxa[1][1:8,:,:,:,:,:] .== j]) .&& idxa[1][1:8,:,:,:,:,:] .== j)

18


8-element Vector{CartesianIndex{6}}:
 CartesianIndex(8, 5, 3, 2, 1, 2)
 CartesianIndex(8, 5, 3, 2, 2, 2)
 CartesianIndex(8, 5, 2, 1, 1, 3)
 CartesianIndex(8, 5, 1, 1, 1, 4)
 CartesianIndex(8, 5, 2, 1, 1, 4)
 CartesianIndex(8, 5, 1, 2, 1, 4)
 CartesianIndex(8, 5, 2, 2, 1, 4)
 CartesianIndex(8, 5, 1, 1, 1, 5)

#### increasing F by 1

In [69]:
# cost of increasing F by 1
# excluding when cost > 100
base_cost = scores[:,1:8,:,:,:,:];
end_cost = scores[:,2:9,:,:,:,:]
diff = end_cost - base_cost
diff[end_cost .> 100] .= -1;

In [72]:
for j in 1:8
    print(maximum(diff[idxa[2][:,1:8,:,:,:,:] .== j]))
    print(" ")
end

20 23 27 34 34 28 20 17 

In [73]:
for j in 1:8
    print(sum(diff[idxa[2][:,1:8,:,:,:,:] .== j] .== maximum(diff[idxa[2][:,1:8,:,:,:,:] .== j])))
    print(" ")
end

71 8 49 6 9 1 4 1 

```
Linearized:

F    =  1  2  3  4
cost =  22 26 30 34 

F    =  5  6  7  8
cost =  34 29 24 19

```

#### increasing L by 1

In [74]:
# cost of increasing L by 1
# excluding when cost > 100
base_cost = scores[:,:,1:8,:,:,:];
end_cost = scores[:,:,2:9,:,:,:]
diff = end_cost - base_cost
diff[end_cost .> 100] .= -1;

In [75]:
maximum(diff)

28

In [76]:
findall(diff .== maximum(diff))

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(9, 1, 4, 8, 5, 3)
 CartesianIndex(9, 1, 4, 6, 7, 3)
 CartesianIndex(9, 1, 4, 7, 7, 3)
 CartesianIndex(9, 1, 4, 6, 8, 3)
 CartesianIndex(9, 1, 4, 9, 3, 4)
 CartesianIndex(9, 1, 4, 9, 4, 4)
 CartesianIndex(9, 1, 4, 7, 5, 4)
 CartesianIndex(9, 1, 4, 4, 8, 4)
 CartesianIndex(9, 1, 4, 8, 2, 5)
 CartesianIndex(9, 1, 4, 4, 6, 5)

In [78]:
[rand(findall(diff .== 25)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(7, 1, 4, 6, 6, 6)
 CartesianIndex(9, 1, 4, 2, 2, 6)
 CartesianIndex(8, 1, 4, 2, 8, 7)
 CartesianIndex(8, 1, 4, 5, 4, 8)
 CartesianIndex(8, 1, 4, 5, 8, 3)
 CartesianIndex(8, 1, 4, 3, 8, 5)
 CartesianIndex(7, 1, 4, 8, 5, 8)
 CartesianIndex(8, 1, 4, 9, 5, 2)
 CartesianIndex(9, 2, 4, 9, 2, 3)
 CartesianIndex(9, 1, 4, 2, 7, 2)

In [79]:
[rand(findall(diff .== 20)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(7, 3, 4, 9, 3, 3)
 CartesianIndex(6, 2, 4, 6, 1, 9)
 CartesianIndex(6, 1, 4, 7, 6, 3)
 CartesianIndex(8, 2, 4, 1, 2, 2)
 CartesianIndex(8, 2, 3, 8, 7, 7)
 CartesianIndex(9, 1, 2, 6, 9, 8)
 CartesianIndex(6, 1, 5, 7, 7, 5)
 CartesianIndex(5, 3, 4, 5, 7, 8)
 CartesianIndex(6, 4, 5, 4, 6, 2)
 CartesianIndex(6, 2, 5, 5, 7, 4)

In [80]:
[rand(findall(diff .== 10)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(5, 1, 7, 3, 4, 1)
 CartesianIndex(6, 6, 3, 1, 5, 3)
 CartesianIndex(6, 3, 2, 2, 7, 2)
 CartesianIndex(5, 4, 2, 7, 8, 8)
 CartesianIndex(4, 3, 4, 2, 4, 3)
 CartesianIndex(4, 3, 7, 2, 8, 3)
 CartesianIndex(6, 3, 2, 3, 1, 5)
 CartesianIndex(5, 5, 3, 8, 3, 3)
 CartesianIndex(4, 1, 6, 3, 7, 3)
 CartesianIndex(6, 6, 3, 2, 7, 1)

#### increasing P by 1

In [81]:
# cost of increasing P by 1
# excluding when cost > 100
base_cost = scores[:,:,:,1:8,:,:];
end_cost = scores[:,:,:,2:9,:,:]
diff = end_cost - base_cost
diff[end_cost .> 100] .= -1;

In [82]:
maximum(diff)

22

In [83]:
findall(diff .== 22)

7-element Vector{CartesianIndex{6}}:
 CartesianIndex(1, 9, 4, 3, 9, 1)
 CartesianIndex(1, 9, 7, 3, 9, 1)
 CartesianIndex(1, 9, 2, 4, 9, 1)
 CartesianIndex(1, 9, 3, 4, 9, 1)
 CartesianIndex(1, 9, 5, 4, 9, 1)
 CartesianIndex(1, 9, 6, 4, 9, 1)
 CartesianIndex(1, 9, 7, 4, 9, 1)

In [85]:
[rand(findall(diff .== 18)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(1, 8, 5, 5, 8, 1)
 CartesianIndex(1, 9, 9, 1, 8, 1)
 CartesianIndex(1, 8, 5, 4, 9, 2)
 CartesianIndex(1, 9, 2, 1, 9, 2)
 CartesianIndex(1, 9, 9, 3, 1, 9)
 CartesianIndex(1, 8, 1, 2, 9, 1)
 CartesianIndex(1, 7, 6, 5, 9, 1)
 CartesianIndex(1, 8, 9, 3, 9, 2)
 CartesianIndex(2, 9, 8, 3, 8, 1)
 CartesianIndex(1, 7, 5, 4, 9, 1)

In [86]:
[rand(findall(diff .== 10)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(2, 9, 6, 6, 4, 1)
 CartesianIndex(1, 7, 4, 7, 2, 5)
 CartesianIndex(1, 8, 8, 6, 4, 3)
 CartesianIndex(4, 6, 5, 1, 9, 3)
 CartesianIndex(1, 5, 4, 4, 3, 7)
 CartesianIndex(1, 5, 8, 8, 1, 7)
 CartesianIndex(1, 6, 9, 8, 6, 2)
 CartesianIndex(1, 7, 9, 6, 5, 2)
 CartesianIndex(2, 9, 3, 2, 5, 4)
 CartesianIndex(4, 5, 4, 5, 4, 7)

#### increasing R by 1

In [87]:
# cost of increasing R by 1
# excluding when cost > 100
base_cost = scores[:,:,:,:,1:8,:];
end_cost = scores[:,:,:,:,2:9,:]
diff = end_cost - base_cost
diff[end_cost .> 100] .= -1;
maximum(diff)

22

In [88]:
findall(diff .== 22)

1-element Vector{CartesianIndex{6}}:
 CartesianIndex(1, 9, 9, 9, 3, 1)

In [89]:
[rand(findall(diff .== 18)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(3, 9, 7, 9, 2, 1)
 CartesianIndex(2, 9, 1, 9, 3, 1)
 CartesianIndex(1, 9, 5, 1, 2, 9)
 CartesianIndex(1, 8, 6, 9, 3, 2)
 CartesianIndex(2, 9, 2, 9, 3, 1)
 CartesianIndex(1, 9, 6, 8, 1, 1)
 CartesianIndex(1, 9, 9, 1, 2, 9)
 CartesianIndex(1, 9, 3, 8, 4, 1)
 CartesianIndex(1, 9, 6, 9, 2, 2)
 CartesianIndex(1, 8, 2, 1, 3, 9)

In [90]:
[rand(findall(diff .== 10)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(1, 6, 3, 5, 4, 6)
 CartesianIndex(4, 6, 5, 2, 3, 7)
 CartesianIndex(1, 6, 4, 2, 1, 8)
 CartesianIndex(3, 7, 5, 2, 5, 5)
 CartesianIndex(4, 5, 1, 2, 7, 9)
 CartesianIndex(2, 7, 3, 7, 2, 2)
 CartesianIndex(1, 7, 3, 3, 6, 5)
 CartesianIndex(2, 9, 5, 1, 7, 5)
 CartesianIndex(4, 5, 3, 3, 3, 9)
 CartesianIndex(4, 4, 5, 2, 5, 7)

#### increasing S by 1

In [94]:
# cost of increasing S by 1
# excluding when cost > 100
base_cost = scores[:,:,:,:,:,1:8];
end_cost = scores[:,:,:,:,:,2:9]
diff = end_cost - base_cost
diff[end_cost .> 100] .= -1;
maximum(diff)

23

In [95]:
findall(diff .== 23)

5-element Vector{CartesianIndex{6}}:
 CartesianIndex(1, 9, 1, 1, 9, 4)
 CartesianIndex(2, 9, 1, 1, 9, 4)
 CartesianIndex(1, 9, 2, 1, 9, 4)
 CartesianIndex(1, 9, 3, 1, 9, 4)
 CartesianIndex(1, 9, 4, 1, 9, 4)

In [96]:
[rand(findall(diff .== 18)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(1, 8, 5, 9, 1, 2)
 CartesianIndex(1, 8, 5, 2, 8, 4)
 CartesianIndex(1, 7, 7, 1, 8, 5)
 CartesianIndex(2, 7, 6, 9, 1, 3)
 CartesianIndex(1, 9, 7, 2, 9, 2)
 CartesianIndex(1, 9, 5, 1, 8, 1)
 CartesianIndex(1, 6, 7, 9, 2, 5)
 CartesianIndex(1, 9, 3, 8, 1, 4)
 CartesianIndex(1, 6, 9, 8, 1, 6)
 CartesianIndex(4, 7, 5, 9, 1, 2)

In [98]:
[rand(findall(diff .== 10)) for i in 1:10]

10-element Vector{CartesianIndex{6}}:
 CartesianIndex(3, 6, 3, 8, 4, 1)
 CartesianIndex(5, 9, 1, 2, 4, 2)
 CartesianIndex(7, 5, 1, 8, 5, 2)
 CartesianIndex(2, 8, 8, 6, 4, 1)
 CartesianIndex(2, 8, 4, 4, 1, 6)
 CartesianIndex(1, 6, 9, 4, 1, 7)
 CartesianIndex(2, 6, 5, 3, 3, 5)
 CartesianIndex(2, 8, 5, 6, 2, 1)
 CartesianIndex(1, 7, 2, 3, 6, 3)
 CartesianIndex(1, 5, 1, 4, 4, 6)

In [103]:
scores[1, 9, 1, 1, 9, 4]

66

In [99]:
scores[1, 9, 2, 1, 9, 4]

67

In [101]:
scores[1, 9, 3, 1, 9, 4]

68

In [102]:
scores[1, 9, 4, 1, 9, 4]

70

### mixed +1/-1 parts

In [112]:
# cost of L+1, S-1
# excluding when end cost > 100 or start cost < 30
base_cost = scores[:,:,1:8,:,:,2:9];
end_cost =  scores[:,:,2:9,:,:,1:8]
diff = end_cost - base_cost
diff[end_cost .> 100] .= -1;
diff[base_cost .< 30] .= -1;
println(maximum(diff))
findall(diff .== maximum(diff))

25


2-element Vector{CartesianIndex{6}}:
 CartesianIndex(9, 1, 4, 7, 8, 2)
 CartesianIndex(9, 1, 4, 7, 7, 3)

In [113]:
# cost of P+1, F-1
# excluding when end cost > 100 or start cost < 30
base_cost = scores[:,2:9,:,1:8,:,:];
end_cost =  scores[:,1:8,:,2:9,:,:]
diff = end_cost - base_cost
diff[end_cost .> 100] .= -1;
diff[base_cost .< 30] .= -1;
println(maximum(diff))
findall(diff .== maximum(diff))

15


5-element Vector{CartesianIndex{6}}:
 CartesianIndex(2, 8, 7, 1, 9, 1)
 CartesianIndex(2, 8, 8, 1, 9, 1)
 CartesianIndex(2, 8, 9, 1, 9, 1)
 CartesianIndex(1, 8, 5, 2, 9, 1)
 CartesianIndex(1, 8, 9, 2, 9, 1)

In [114]:
scores[2, 9, 7, 1, 9, 1]

31

In [115]:
scores[2, 8, 7, 2, 9, 1]

46

In [119]:
# cost of P+2, R+2, F-1, L-1
# excluding when end cost > 100 or start cost < 30
base_cost = scores[:,2:9,2:9,1:7,1:7,:];
end_cost =  scores[:,1:8,1:8,3:9,3:9,:]
diff = end_cost - base_cost
diff[end_cost .> 100] .= -1;
diff[base_cost .< 30] .= -1;
println(maximum(diff))
findall(diff .== maximum(diff))

52


7-element Vector{CartesianIndex{6}}:
 CartesianIndex(1, 8, 1, 1, 1, 9)
 CartesianIndex(1, 8, 2, 1, 1, 9)
 CartesianIndex(1, 8, 3, 1, 1, 9)
 CartesianIndex(1, 8, 5, 1, 1, 9)
 CartesianIndex(1, 8, 6, 1, 1, 9)
 CartesianIndex(1, 8, 7, 1, 1, 9)
 CartesianIndex(1, 8, 8, 1, 1, 9)

In [120]:
scores[1, 9, 2, 1, 1, 9]

30

In [121]:
scores[1, 8, 1, 3, 3, 9]

82

## Assigning simple costs to random part sets, up to 3 parts (base + 2 mods)

A simple cost doesn't depend on base stats

In [214]:
n_base_parts = 20
min_cost_base = 40
max_cost_base = 70
min_cost_mod = 10
max_cost_mod = 40
parts = Array{Int64}(undef, (n_base_parts, 7))
for i_base in 1:20
    desired_cost = rand(min_cost_base:max_cost_base)
    indx = rand(findall(scores .== desired_cost))
    for j in 1:6
        parts[i_base, j] = indx[j]
    end
    parts[i_base, 7] = desired_cost
end

In [215]:
# generate completely random mods
n_mods0 = 60
mods = Array{Int64}(undef, (n_mods0, 7))
min_delt = -2
max_delt = 4
for i_mod in 1:n_mods0
    v = [0,0,0,0,0,0]
    for j in 1:3
        v[rand(1:6)] = rand(min_delt:max_delt)
    end
    mods[i_mod, 1:6] = v
    mods[i_mod, 7] = 0
end

mods;

In [216]:
# compute max benefit of single mod
for i_mod in 1:n_mods0
    v = mods[i_mod, 1:6]
    max_diff = 0
    for i_base in 1:n_base_parts
        b = parts[i_base, 1:6]
        b2 = b .+ v
        if minimum(b2) >= 1 && maximum(b2) <= 9
            cost2 = scores[CartesianIndex((b2...))]
            diff = cost2 - parts[i_base, 7]
            if diff > max_diff
                max_diff = diff
            end
        end
    end
    mods[i_mod, 7] = max_diff
end

In [217]:
mods = mods[mods[:, 7] .<= max_cost_mod, :]
n_mods1 = size(mods)[1]
mods

29×7 Matrix{Int64}:
  0   0  -1   0   3   0  36
  0   0  -2   0  -1   1   0
  1  -2   0   0   0   1  31
  0   0   0   0  -2   2   9
  0  -1   0   3   0   0  29
  0   0  -2   0   0   0   0
  0  -2  -2   0   2   0   2
  0  -2   0   0   3   0  16
  0   0   0   0   1   0  12
  1   0  -2   0  -1   0  17
  0   0  -1   0  -1   0   0
  0   0   1  -2   0   0  15
 -2   0   3   0   0   0  35
  ⋮                   ⋮  
  0   0   0  -2   0  -2   0
  0   0  -1   0   2   0  23
  0   0   0   1   0  -1   1
  1   1  -1   0   0   0  39
  0   0   0  -1  -2   3  12
  0   0   0   2   0   0  26
  0   0   0   0   3   0  38
  0   0   0  -1   0   1   6
  0   0   0   0   3  -2  10
  0  -1   0   0  -2   0   0
 -1   0   0   2   0   0  17
  1  -1   0   0   0   0  27

In [219]:
flag = true
while flag
    # check if there are 2-combos that are undercosted
    mod_ids = Array{Int64}(undef, 100000)
    diffs = Array{Int64}(undef, 100000)
    u_i = 0
    for i1 in 1:(n_mods1 - 1)
        v1 = mods[i1, 1:6]
        c1 = mods[i1, 7]
        for i2 in (i1+1):n_mods1
            v2 = mods[i2, 1:6]
            v3 = v1 .+ v2
            c2 = mods[i2, 7]
            for i_base in 1:n_base_parts
                b = parts[i_base, 1:6]
                c = parts[i_base, 7] + c1 + c2
                path1 = minimum(b .+ v1) >=1 && maximum(b .+ v1) <= 9 && minimum(b .+ v3) >= 1 && maximum(b .+ v3) <= 9
                path2 = minimum(b .+ v2) >=1 && maximum(b .+ v2) <= 9 && minimum(b .+ v3) >= 1 && maximum(b .+ v3) <= 9
                if path1 || path2
                    newc = scores[CartesianIndex((b.+v3)...)]
                    if newc > c
                        u_i += 1
                        mod_ids[u_i] = i1
                        diffs[u_i] = newc - c
                        u_i += 1
                        mod_ids[u_i] = i2
                        diffs[u_i] = newc - c
                    end
                end
            end
        end
    end
    diffs = diffs[1:u_i]
    mod_ids = mod_ids[1:u_i];
    if length(diffs) == 0
        flag = false
    else
        mids = sort(unique(mod_ids))
        blames = [mean(diffs[mod_ids .== m])/2 for m in mids]
        blames = map(x -> convert(Int64, floor(x)), blames)
        blames[blames .< 1] .= 1
        mods[mids, 7] = mods[mids, 7] + blames        
    end
end

In [220]:
mods = mods[mods[:, 7] .>= min_cost_mod .&& mods[:, 7] .<= max_cost_mod, :]
n_mods = size(mods)[1]

19

In [221]:
mods

19×7 Matrix{Int64}:
  0   0  -1   0   3   0  38
  1  -2   0   0   0   1  33
  0   0   0   0  -2   2  10
  0  -1   0   3   0   0  30
  0  -2   0   0   3   0  16
  0   0   0   0   1   0  13
  1   0  -2   0  -1   0  17
  0   0   1  -2   0   0  15
 -2   0   3   0   0   0  35
  0   0  -2   4   0  -2  19
  0   0  -2  -1   4   0  37
  0   0  -1   0   2   0  24
  1   1  -1   0   0   0  39
  0   0   0  -1  -2   3  13
  0   0   0   2   0   0  26
  0   0   0   0   3   0  39
  0   0   0   0   3  -2  14
 -1   0   0   2   0   0  19
  1  -1   0   0   0   0  29

In [222]:
# Generate Spuds
library = Array{Spud}(undef, n_base_parts * n_mods * n_mods)
ff_i = 0
for i1 in 1:(n_mods - 1)
    v1 = mods[i1, 1:6]
    c1 = mods[i1, 7]
    for i2 in (i1+1):n_mods
        v2 = mods[i2, 1:6]
        c2 = mods[i2, 7]
        v3 = v1 .+ v2
        for i_base in 1:n_base_parts
            b = parts[i_base, 1:6]
            c = parts[i_base, 7] + c1 + c2
            v = b .+ v3
            path1 = minimum(b .+ v1) >=1 && maximum(b .+ v1) <= 9 && minimum(b .+ v3) >= 1 && maximum(b .+ v3) <= 9
            path2 = minimum(b .+ v2) >=1 && maximum(b .+ v2) <= 9 && minimum(b .+ v3) >= 1 && maximum(b .+ v3) <= 9
            if c <= 100 && (path1 || path2)
                ff_i += 1
                if path1
                    nm = string(i_base, ".", i1, ".", i2, "(", c, ")")
                else
                    nm = string(i_base, ".", i2, ".", i1, "(", c, ")")
                end
                library[ff_i] = Spud(nm, v[1],v[2],v[3],v[4],v[5],v[6],999)
            end
        end
    end
end
library = library[1:ff_i]

752-element Vector{Spud}:
 Spud("1.1.3(99)", 1, 9, 4, 3, 3, 8, 999)
 Spud("2.1.3(96)", 3, 8, 6, 1, 3, 6, 999)
 Spud("4.1.3(90)", 5, 4, 2, 2, 5, 9, 999)
 Spud("6.1.3(96)", 4, 4, 4, 7, 7, 5, 999)
 Spud("11.1.3(88)", 8, 2, 1, 2, 4, 9, 999)
 Spud("16.3.1(100)", 5, 1, 5, 1, 8, 6, 999)
 Spud("18.1.3(88)", 4, 4, 1, 9, 7, 5, 999)
 Spud("19.1.3(95)", 3, 6, 7, 1, 2, 8, 999)
 Spud("2.1.6(99)", 3, 8, 6, 1, 6, 4, 999)
 Spud("4.1.6(93)", 5, 4, 2, 2, 8, 7, 999)
 Spud("11.1.6(91)", 8, 2, 1, 2, 7, 7, 999)
 Spud("19.1.6(98)", 3, 6, 7, 1, 5, 6, 999)
 Spud("8.8.1(96)", 6, 4, 1, 4, 8, 4, 999)
 ⋮
 Spud("11.17.19(83)", 9, 1, 2, 2, 6, 5, 999)
 Spud("15.17.19(98)", 5, 3, 9, 1, 5, 2, 999)
 Spud("18.17.19(83)", 5, 3, 2, 9, 9, 1, 999)
 Spud("19.17.19(90)", 4, 5, 8, 1, 4, 4, 999)
 Spud("1.19.18(99)", 1, 8, 5, 5, 2, 6, 999)
 Spud("2.18.19(96)", 3, 7, 7, 3, 2, 4, 999)
 Spud("4.18.19(90)", 5, 3, 3, 4, 4, 7, 999)
 Spud("6.18.19(96)", 4, 3, 5, 9, 6, 3, 999)
 Spud("8.18.19(91)", 6, 3, 1, 8, 5, 4, 999)
 Spud("10.18.19(91

In [224]:
@time counts = ffp(library, 10000);

  0.507475 seconds (8.35 M allocations: 842.883 MiB, 20.12% gc time)


In [227]:
spuds_to_df(library[counts .> 1])

Row,name,h,f,l,p,r,s,a1
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,6.2.8(96),5,2,6,5,6,4,999
2,20.3.8(93),9,3,4,4,4,3,999
3,4.3.13(91),6,5,2,2,2,9,999
4,5.3.17(88),2,7,1,3,8,4,999
5,12.3.17(82),3,5,6,8,7,1,999
6,13.17.3(91),2,5,5,6,3,8,999
7,1.4.5(97),1,6,5,6,5,6,999
8,6.14.4(91),4,3,5,9,4,6,999
9,9.5.6(97),3,1,7,7,6,8,999
